In [ ]:
import yfinance as yf 
from yfinance import EquityQuery
import numpy as np
import pandas as pd
import time

size = 20
rsi_window = 14
roc_window = 5
volatility_window = 30

In [27]:
from yfinance import EquityQuery, screen
import pandas as pd

# Define your screen query
screen_query = EquityQuery("and", [
    EquityQuery("is-in", ["region", "us"]),
    EquityQuery("is-in", ["sector",
        "Technology",
        "Financial Services",
        "Consumer Cyclical",
        "Communication Services",
        "Basic Materials",
        "Industrials"
    ]),
    EquityQuery("GTE", ["eodprice", 0.50]),
    EquityQuery("GTE", ["avgdailyvol3m", 100000]),
    EquityQuery("LTE", ["short_percentage_of_float.value", 100])
])

batch_size = size  # Now set to 100
offset = 0
all_stocks = []
setno = offset / batch_size + 1
while True:
    screener = screen(
        screen_query,
        sortField="avgdailyvol3m",
        sortAsc=False,
        size=batch_size,
        offset=offset
    )
    stocks = screener.get('quotes', [])
    if not stocks:
        break

    all_stocks.extend(stocks)
    offset += batch_size
    print(f"Loaded {len(all_stocks)} stocks")

    # Prompt user to continue
    user_input = input("Load next stocks? (y/n): ")
    if user_input.lower() != 'y':
        break

# Convert to DataFrame
stocks = pd.DataFrame(all_stocks)
print(f"Total stocks loaded: {len(stocks)}")


Loaded 5 stocks
Loaded 10 stocks
Total stocks loaded: 10


In [28]:
stocks = pd.DataFrame(stocks)
stocks = stocks.drop(columns=[
    'esgPopulated',
    'cryptoTradeable',
    'hasPrePostMarketData',
    'firstTradeDateMilliseconds',
    'priceHint',
    'regularMarketDayHigh',
    'regularMarketDayRange',
    'regularMarketDayLow',
    'regularMarketPreviousClose',
    'messageBoardId',
    'fullExchangeName',
    'financialCurrency',
    'regularMarketOpen',
    'averageDailyVolume3Month',
    'averageDailyVolume10Day',
    'corporateActions',
    'fiftyTwoWeekLowChange',
    'fiftyTwoWeekLowChangePercent',
    'fiftyTwoWeekRange',
    'fiftyTwoWeekHighChange',
    'fiftyTwoWeekHighChangePercent',
    'fiftyTwoWeekChangePercent',
    'earningsTimestamp',
    'earningsTimestampStart',
    'earningsTimestampEnd',
    'earningsCallTimestampStart',
    'earningsCallTimestampEnd',
    'dividendRate',
    'trailingAnnualDividendYield',
    'marketState',
    'epsCurrentYear',
    'fiftyDayAverage',
    'fiftyDayAverageChange',
    'fiftyDayAverageChangePercent',
    'twoHundredDayAverage',
    'twoHundredDayAverageChange',
    'twoHundredDayAverageChangePercent','dividendYield','gmtOffSetMilliseconds','bid','ask','currency', 'quoteType','language','typeDisp','quoteSourceName','triggerable','customPriceAlertConfidence','sourceInterval','exchangeDataDelayedBy','exchangeTimezoneName','exchangeTimezoneShortName',
 'bidSize','askSize','market','isEarningsDateEstimate','trailingAnnualDividendRate','regularMarketTime','priceEpsCurrentYear','epsForward','forwardPE','tradeable','displayName','longName'
]
)
df = stocks

In [29]:
order = [
    # Identification
    'symbol',
    'shortName',
    'exchange',
    'region',
    
    # Market Price & Change
    'regularMarketPrice',
    'regularMarketChange',
    'regularMarketChangePercent',
    'regularMarketVolume',
    
    # Valuation Metrics
    'marketCap',
    'sharesOutstanding',
    'bookValue',
    'priceToBook',
    'trailingPE',
    'epsTrailingTwelveMonths',
    'averageAnalystRating',
    
    # Price Range
    'fiftyTwoWeekHigh',
    'fiftyTwoWeekLow',
    
    # Dividend Info
    'dividendDate'
]
df = df[order]
df


,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,bookValue,priceToBook,trailingPE,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate
0,NVDA,NVIDIA Corporation,NMS,US,135.1300,-4.060000,-2.916870,323313507,3295496634368,24387600384,3.438,39.304830,43.731396,3.09,1.4 - Strong Buy,153.13,86.620,1.743552e+09
1,F,Ford Motor Company,NYQ,US,10.3800,0.160000,1.565560,132007518,41276588032,3905700096,11.225,0.924722,8.304000,1.25,3.0 - Hold,14.85,8.440,1.748822e+09
2,TSLA,"Tesla, Inc.",NMS,US,346.4600,-11.970000,-3.339560,122775188,1115933769728,3220960000,23.184,14.943927,195.740110,1.77,2.6 - Hold,488.54,167.410,NaN
3,LCID,"Lucid Group, Inc.",NMS,US,2.2300,-0.180000,-7.468880,254714279,6802079744,3050259968,1.044,2.136015,NaN,-1.19,3.1 - Hold,4.43,1.930,NaN
4,PLTR,Palantir Technologies Inc.,NMS,US,131.7800,9.460000,7.733810,182153344,310988931072,2262909952,2.299,57.320570,599.000000,0.22,3.1 - Hold,133.49,21.005,NaN
5,PLUG,"Plug Power, Inc.",NCM,US,0.8827,-0.043700,-4.717180,145939202,951850688,1078339968,1.900,0.464579,NaN,-2.43,2.9 - Hold,3.55,0.690,NaN
6,AGMH,AGM Group Holdings Inc.,NCM,US,0.0600,-0.008000,-11.764700,11617547,5922840,98714000,0.901,0.066593,0.428571,0.14,NaN,2.19,0.022,NaN
7,INTC,Intel Corporation,NMS,US,19.5500,-0.700001,-3.456790,112628808,85277097984,4361999872,22.869,0.854869,NaN,-4.48,3.0 - Hold,37.16,17.670,1.725149e+09
8,AAL,"American Airlines Group, Inc.",NMS,US,11.4100,0.010000,0.087721,62864216,7525031936,659512000,-6.836,-1.669105,11.410000,1.00,2.1 - Buy,19.10,8.500,1.582070e+09
9,AAPL,Apple Inc.,NMS,US,200.8500,0.900009,0.450117,69799163,2999855611904,14935799808,4.471,44.922836,31.236393,6.43,2.1 - Buy,260.10,169.210,1.747267e+09


In [30]:
# Get the price history for the stocks, to do RSI, or MACD, options volatility, and kalman filter acceleration(or see tview workaround)
stocklist = df.symbol.tolist()
stocklist

# Get the price history for the stocks
price_data = yf.download(stocklist, period='14d', interval='1d')['Close']


# Calculate the daily returns
change_data = price_data[stocklist]/price_data[stocklist].shift(1)-1

# RSI
change_data
def rsi(price_data, period=rsi_window):
    delta = change_data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))
rsi_data = rsi(price_data)
rsi_data

rsi = round(rsi_data.iloc[-1],1)
df['RSI'] = rsi.values


## Rate of change 
window=5 
recent_avg = change_data.rolling(window=roc_window).mean()
earlier_avg = change_data.shift(window).rolling(window=roc_window).mean()

# Acceleration: difference between recent and earlier averages
accel = (recent_avg - earlier_avg) * 100
accel = round(accel.iloc[-1], 2)
df['ROC'] = accel.values


# Apply filters________________________________________________________________________________________________________________________________________________________

filterdf = df[(df['RSI'] < 60) ] # Filter for RSI < 60

filterdf



[*********************100%***********************]  10 of 10 completed
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_53652/3717644340.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI'] = rsi.values
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_53652/3717644340.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ROC'] = accel.values


,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,bookValue,priceToBook,trailingPE,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate,RSI,ROC
0,NVDA,NVIDIA Corporation,NMS,US,135.1300,-4.060000,-2.916870,323313507,3295496634368,24387600384,3.438,39.304830,43.731396,3.09,1.4 - Strong Buy,153.13,86.620,1.743552e+09,36.6,0.67
1,F,Ford Motor Company,NYQ,US,10.3800,0.160000,1.565560,132007518,41276588032,3905700096,11.225,0.924722,8.304000,1.25,3.0 - Hold,14.85,8.440,1.748822e+09,55.3,0.47
2,TSLA,"Tesla, Inc.",NMS,US,346.4600,-11.970000,-3.339560,122775188,1115933769728,3220960000,23.184,14.943927,195.740110,1.77,2.6 - Hold,488.54,167.410,NaN,41.6,0.46
3,LCID,"Lucid Group, Inc.",NMS,US,2.2300,-0.180000,-7.468880,254714279,6802079744,3050259968,1.044,2.136015,NaN,-1.19,3.1 - Hold,4.43,1.930,NaN,44.9,-3.37
4,PLTR,Palantir Technologies Inc.,NMS,US,131.7800,9.460000,7.733810,182153344,310988931072,2262909952,2.299,57.320570,599.000000,0.22,3.1 - Hold,133.49,21.005,NaN,49.5,2.46
5,PLUG,"Plug Power, Inc.",NCM,US,0.8827,-0.043700,-4.717180,145939202,951850688,1078339968,1.900,0.464579,NaN,-2.43,2.9 - Hold,3.55,0.690,NaN,51.9,-0.64
6,AGMH,AGM Group Holdings Inc.,NCM,US,0.0600,-0.008000,-11.764700,11617547,5922840,98714000,0.901,0.066593,0.428571,0.14,NaN,2.19,0.022,NaN,48.2,-8.53
7,INTC,Intel Corporation,NMS,US,19.5500,-0.700001,-3.456790,112628808,85277097984,4361999872,22.869,0.854869,NaN,-4.48,3.0 - Hold,37.16,17.670,1.725149e+09,41.8,-0.03
8,AAL,"American Airlines Group, Inc.",NMS,US,11.4100,0.010000,0.087721,62864216,7525031936,659512000,-6.836,-1.669105,11.410000,1.00,2.1 - Buy,19.10,8.500,1.582070e+09,43.3,0.64
9,AAPL,Apple Inc.,NMS,US,200.8500,0.900009,0.450117,69799163,2999855611904,14935799808,4.471,44.922836,31.236393,6.43,2.1 - Buy,260.10,169.210,1.747267e+09,48.4,0.94


#"""Steps:
#1. get options volatility, prices, data 
#2. get RSI, kalman filter acceleration  (done)
#indicators in screener: filter by RSI, filter by relative options volatility (vs historical volatility, as yfinance does not have historical options volatility data)
#"""
# other:
#take steps to pull all yahoo screen companies 

In [31]:
filterstocks = filterdf.symbol.tolist()
price_datayear = yf.download(filterstocks, period='3mo', interval='1d')["Close"]
#^^^ fix 
# Historical volatiltiy calculation
# Next add the vol smile relative to HV 
def hv(price_datayear, window=volatility_window):
    log_returns = np.log(price_datayear / price_datayear.shift(1))
    volatility = log_returns.rolling(window=window).std() * np.sqrt(252)  # Annualized volatility
    return volatility

hvol = hv(price_datayear)
hvol

[*********************100%***********************]  10 of 10 completed


Ticker,AAL,AAPL,AGMH,F,INTC,LCID,NVDA,PLTR,PLUG,TSLA
Date,,,,,,,,,,
2025-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2025-05-23,0.417077,0.356372,4.273777,0.259747,0.446677,0.660853,0.452937,0.704987,1.206825,0.548100
2025-05-27,0.427492,0.344716,4.273939,0.256640,0.451565,0.666045,0.453315,0.704838,1.207457,0.571533
2025-05-28,0.431043,0.338702,4.270426,0.237399,0.444981,0.665529,0.453923,0.697767,1.298412,0.576639


In [32]:
filterstocks = filterdf.symbol.tolist()
#  1. Download price data (make sure columns are tickers)
price_datayear = yf.download(filterstocks, period='3mo', interval='1d')['Close']

# 2. Calculate HV
hvol_latest = price_datayear.apply(hv).iloc[-1]
hvol_latest.name = 'hvol'

# 3. Prepare for merge
filterdf['symbol'] = filterdf['symbol'].astype(str).str.upper()
hvol_latest.index = hvol_latest.index.astype(str).str.upper()

# 4. Merge
filterdf = filterdf.merge(hvol_latest, left_on='symbol', right_index=True, how='left')


[*********************100%***********************]  10 of 10 completed


In [ ]:
import yfinance as yf
import pandas as pd

def find_closest_value(data_list, target_value):
    """Find the value in a list or Series that is closest to a given target value."""
    if len(data_list) == 0:
        return None
    return min(data_list, key=lambda x: abs(x - target_value))

def filter_special_expiry(expiry_list):
    """
    Returns a list of expiries filtered to only the 2nd lowest, middle, 3rd to last, and last.
    """
    sorted_expiries = sorted(expiry_list)
    length = len(sorted_expiries)
    if length == 0:
        return []
    values_to_keep = set()
    if length > 1:
        values_to_keep.add(sorted_expiries[1])  # 2nd lowest
    values_to_keep.add(sorted_expiries[length // 2])  # middle
    if length >= 3:
        values_to_keep.add(sorted_expiries[-3])  # 3rd to last
    values_to_keep.add(sorted_expiries[-1])  # last
    return list(values_to_keep)

def get_options(symbol):
    try:
        stock = yf.Ticker(symbol)
        options_chain = stock.options
        if not options_chain:
            return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest','symbol'])
        # Filter expiries first
        filtered_expiries = filter_special_expiry(options_chain)
        options_data = []
        current_price = stock.history(period='1d')['Close'].iloc[-1]
        for expiry in filtered_expiries:
            chain = stock.option_chain(expiry)
            for opt_type, df in [('call', chain.calls), ('put', chain.puts)]:
                if not df.empty:
                    closest_strike = find_closest_value(df['strike'], current_price)
                    filtered = df[df['strike'] == closest_strike].copy()
                    filtered['type'] = opt_type
                    filtered['expiry'] = expiry
                    filtered['symbol'] = symbol
                    options_data.append(filtered)
                      # To avoid hitting API limits
        if options_data:
            result = pd.concat(options_data, ignore_index=True)
            # Return only strike, impliedVolatility, and openInterest
            return result[['strike', 'impliedVolatility', 'openInterest','symbol','expiry','type']]
        else:
            return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest','symbol','expiry','type'])
    except Exception as e:
        print(f"Error fetching options for {symbol}: {e}")
        return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest','symbol','expiry','type'])





# Example usage:
# filterstocks = ['AAPL', 'MSFT', 'NVDA', ...]
options_list = [get_options(symbol) for symbol in filterstocks] ### Change this to filterstocks to get all options

options = pd.concat(options_list, ignore_index=True)
options = options[options['openInterest']>100]
options

/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_53652/2605090219.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  options = pd.concat(options_list, ignore_index=True)


,strike,impliedVolatility,openInterest,symbol
0,135.0,0.529546,5449.0,NVDA
1,135.0,0.382788,1873.0,NVDA
2,135.0,0.480169,5561.0,NVDA
3,135.0,0.410589,2523.0,NVDA
4,135.0,0.387213,7038.0,NVDA
...,...,...,...,...
67,200.0,0.279060,2266.0,AAPL
68,200.0,0.329841,2981.0,AAPL
69,200.0,0.320197,2480.0,AAPL
70,200.0,0.355765,6692.0,AAPL


In [34]:
## hvol, filterdf, options 
# After filtering for open interest
options = options[options['openInterest'] > 100]





In [35]:
# Ensure 'symbol' exists and options is not empty
if not options.empty and 'symbol' in options.columns and 'impliedVolatility' in options.columns:
    avg_iv = options.groupby('symbol')['impliedVolatility'].mean().reset_index()
    avg_iv.columns = ['symbol', 'avg_iv']
else:
    avg_iv = pd.DataFrame(columns=['symbol', 'avg_iv'])

# Merge with filterdf
filterdf['symbol'] = filterdf['symbol'].astype(str).str.upper()
avg_iv['symbol'] = avg_iv['symbol'].astype(str).str.upper()
filterdf = filterdf.merge(avg_iv, on='symbol', how='left')

# Safe renaming and ratio calculation
if 'hvol' in filterdf.columns:
    filterdf = filterdf.rename(columns={'hvol': 'historical_volatility', 'avg_iv': 'average_implied_volatility'})
    if 'average_implied_volatility' in filterdf.columns and 'historical_volatility' in filterdf.columns:
        filterdf["iv/hv_ratio"] = filterdf["average_implied_volatility"] / filterdf["historical_volatility"]
    else:
        filterdf["iv/hv_ratio"] = None
else:
    filterdf["historical_volatility"] = None
    filterdf["iv/hv_ratio"] = None

# Save and display results
import datetime
today = datetime.datetime.today().strftime('%Y-%m-%d')
filename = f'/Users/nshaffer/Desktop/equity_vol_screen_{today}_offset:{offset}.csv'
filterdf.to_csv(filename, index=False)

filterdf


,symbol,shortName,exchange,region,regularMarketPrice,regularMarketChange,regularMarketChangePercent,regularMarketVolume,marketCap,sharesOutstanding,...,epsTrailingTwelveMonths,averageAnalystRating,fiftyTwoWeekHigh,fiftyTwoWeekLow,dividendDate,RSI,ROC,historical_volatility,average_implied_volatility,iv/hv_ratio
0,NVDA,NVIDIA Corporation,NMS,US,135.1300,-4.060000,-2.916870,323313507,3295496634368,24387600384,...,3.09,1.4 - Strong Buy,153.13,86.620,1.743552e+09,36.6,0.67,0.411135,0.436361,1.061357
1,F,Ford Motor Company,NYQ,US,10.3800,0.160000,1.565560,132007518,41276588032,3905700096,...,1.25,3.0 - Hold,14.85,8.440,1.748822e+09,55.3,0.47,0.222140,0.333259,1.500217
2,TSLA,"Tesla, Inc.",NMS,US,346.4600,-11.970000,-3.339560,122775188,1115933769728,3220960000,...,1.77,2.6 - Hold,488.54,167.410,NaN,41.6,0.46,0.563089,0.611868,1.086627
3,LCID,"Lucid Group, Inc.",NMS,US,2.2300,-0.180000,-7.468880,254714279,6802079744,3050259968,...,-1.19,3.1 - Hold,4.43,1.930,NaN,44.9,-3.37,0.724037,0.879884,1.215247
4,PLTR,Palantir Technologies Inc.,NMS,US,131.7800,9.460000,7.733810,182153344,310988931072,2262909952,...,0.22,3.1 - Hold,133.49,21.005,NaN,49.5,2.46,0.679801,0.640877,0.942742
5,PLUG,"Plug Power, Inc.",NCM,US,0.8827,-0.043700,-4.717180,145939202,951850688,1078339968,...,-2.43,2.9 - Hold,3.55,0.690,NaN,51.9,-0.64,1.275492,1.319339,1.034377
6,AGMH,AGM Group Holdings Inc.,NCM,US,0.0600,-0.008000,-11.764700,11617547,5922840,98714000,...,0.14,NaN,2.19,0.022,NaN,48.2,-8.53,4.282991,NaN,NaN
7,INTC,Intel Corporation,NMS,US,19.5500,-0.700001,-3.456790,112628808,85277097984,4361999872,...,-4.48,3.0 - Hold,37.16,17.670,1.725149e+09,41.8,-0.03,0.442356,0.470312,1.063198
8,AAL,"American Airlines Group, Inc.",NMS,US,11.4100,0.010000,0.087721,62864216,7525031936,659512000,...,1.00,2.1 - Buy,19.10,8.500,1.582070e+09,43.3,0.64,0.400287,0.479701,1.198393
9,AAPL,Apple Inc.,NMS,US,200.8500,0.900009,0.450117,69799163,2999855611904,14935799808,...,6.43,2.1 - Buy,260.10,169.210,1.747267e+09,48.4,0.94,0.317624,0.313554,0.987187
